## READ TEST, TRAIND AND VALIDATION FILES

In [1]:
import pandas as pd
# Read the .tsv file into a Pandas DataFrame
df_train = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/train.tsv', sep='\t')
df_valid = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/validation.tsv', sep='\t')
df_test = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/test.tsv', sep='\t')
# Print the first few rows of the DataFrame

In [2]:
df_train.head()

,file_name,label,shaming,stereotype,objectification,violence,text
0,8716.jpg,1,0,1,1,0,GETS MARRIED TO GIRL OF HIS DREAMS SHE DOESN'T...
1,3066.jpg,1,1,1,0,0,When your mama don't change yo diaper for 19 y...
2,6038.jpg,0,0,0,0,0,"Some people want a big house, a fast car, and ..."
3,10861.jpg,1,0,0,1,0,FAP FAP FAP FAP FAP FAP memecenter Meme Center
4,11198.jpg,0,0,0,0,0,I RAISE. A I CALL. I FOLD. I'M ALL IN


### Attaching the image file_path

In [3]:
import pandas as pd

# Read the .tsv file into a Pandas DataFrame
df_train = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/train.tsv', sep='\t')
df_valid = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/validation.tsv', sep='\t')
df_test = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/test.tsv', sep='\t')

# Define the path to the image folders
train_image_folder = '/kaggle/input/multimodal-misogyny-detection-mami-2022/MAMI_2022_images/training_images/'
test_image_folder = '/kaggle/input/multimodal-misogyny-detection-mami-2022/MAMI_2022_images/test_images/'

# Function to generate file path using file name
def get_file_path(row):
    if row['file_name'] in df_train['file_name'].values:
        return train_image_folder + row['file_name']
    elif row['file_name'] in df_valid['file_name'].values:
        return train_image_folder + row['file_name']
    elif row['file_name'] in df_test['file_name'].values:
        return test_image_folder + row['file_name']
    else:
        return None

# Apply the function to generate file path
df_train['file_path'] = df_train.apply(get_file_path, axis=1)
df_valid['file_path'] = df_valid.apply(get_file_path, axis=1)
df_test['file_path'] = df_test.apply(get_file_path, axis=1)

In [4]:
df_valid.file_path[0]

'/kaggle/input/multimodal-misogyny-detection-mami-2022/MAMI_2022_images/training_images/1377.jpg'

### RUNNING AN INFERENCE ON THE TEST IMAGES using nlpconnect/vit-gpt2-image-captioning
This is to check the model behave on these images

In [5]:
# import pandas as pd
# import torch
# from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
# from PIL import Image

# # Load pre-trained model and tokenizer
# model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# # Move model to available device (GPU or CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Set generation parameters
# max_length = 16
# num_beams = 4
# gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

# # Define function to predict captions for images with attention mask
# def predict_caption(df):
#     captions = []
#     for index, row in df.iterrows():
#         image_path = row['file_path']
#         image = Image.open(image_path)
#         if image.mode != "RGB":
#             image = image.convert(mode="RGB")
        
#         pixel_values = feature_extractor(images=[image], return_tensors="pt").pixel_values
#         pixel_values = pixel_values.to(device)
        
#         # Create attention mask to avoid padding tokens affecting the output
#         attention_mask = torch.ones_like(pixel_values)
#         attention_mask[pixel_values == 0] = 0
        
#         output_ids = model.generate(pixel_values, attention_mask=attention_mask, **gen_kwargs)
#         preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
#         preds = [pred.strip() for pred in preds]
        
#         captions.append(preds[0])  # Assuming only one caption per image
        
#     return captions

Predict and plot only 5 random images


In [6]:
# from PIL import Image
# import random
# import matplotlib.pyplot as plt

# # Select 5 random images from the test set
# random_indices = random.sample(range(len(df_test)), 5)
# df_sampled = df_test.iloc[random_indices]

# # Predict captions for the sampled images
# predicted_captions_sampled = predict_caption(df_sampled)

# # Plot the sampled images with their predicted captions
# plt.figure(figsize=(15, 10))
# for i, (index, row) in enumerate(df_sampled.iterrows()):
#     image_path = row['file_path']
#     image = Image.open(image_path)
    
#     plt.subplot(2, 3, i+1)
#     plt.imshow(image)
#     plt.title(predicted_captions_sampled[i])
#     plt.axis('off')

# plt.tight_layout()
# plt.show()

In [7]:
# # Print the first few rows of the DataFrame with predicted captions
# df_pred.caption[6]

### USING OCR TO READ THE TEXT IN THE IMAGES

In [8]:
# from transformers import TrOCRProcessor, VisionEncoderDecoderModel
# import requests
# from PIL import Image
# import pandas as pd

# def perform_ocr_on_images(df):
#     # Load TrOCR processor and VisionEncoderDecoderModel
#     processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
#     model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
    
#     ocr_results = []
    
#     for index, row in df.iterrows():
#         file_path = row['file_path']
        
#         # Load image from the file path
#         image = Image.open(file_path).convert("RGB")
        
#         # Process the image using the TrOCR processor
#         pixel_values = processor(image, return_tensors="pt").pixel_values
        
#         # Generate text from the processed image using the VisionEncoderDecoderModel
#         generated_ids = model.generate(pixel_values)
#         generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
#         ocr_results.append(generated_text)
    
#     return ocr_results



In [9]:
# from PIL import Image
# import random
# import matplotlib.pyplot as plt

# # Select 5 random images from the test set
# random_indices = random.sample(range(len(df_test)), 5)
# df_sampled = df_test.iloc[random_indices]

# # Predict captions for the sampled images
# predicted_captions_sampled = perform_ocr_on_images(df_sampled)

# # Plot the sampled images with their predicted captions
# plt.figure(figsize=(15, 10))
# for i, (index, row) in enumerate(df_sampled.iterrows()):
#     image_path = row['file_path']
#     image = Image.open(image_path)
    
#     plt.subplot(2, 3, i+1)
#     plt.imshow(image)
#     plt.title(predicted_captions_sampled[i])
#     plt.axis('off')

# plt.tight_layout()
# plt.show()

# IMAGE BASELINE VGG16

In [10]:
batch_size = 32
epochs = 50
image_size = 224
threshold = 0.5


In [14]:
import pandas as pd
# import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import keras
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import regularizers
import os
import gc


In [15]:
# Convert the label column to strings
df_train["label"] = df_train["label"].astype(str)
df_test["label"] = df_test["label"].astype(str)
df_valid["label"] = df_valid["label"].astype(str)

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data generator for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest',
    
)

# Define the data generator for validation images
valid_datagen = ImageDataGenerator(rescale=1./255)

# Define the batch size
batch_size = 32

# Create the training data generator
train_dataset = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='file_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Create the validation data generator
val_dataset  = valid_datagen.flow_from_dataframe(
    dataframe=df_valid,
    x_col='file_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
)

# Define the data generator for test images
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the test data generator
test_dataset = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='file_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
)


# # Define the data generator for test images
# test_datagen = ImageDataGenerator(rescale=1./255)

# # Create the test data generator
# test_generator = test_datagen.flow_from_dataframe(
#     dataframe=df_test,
#     x_col='roi_path',
#     y_col='label',
#     target_size=(224, 224),
#     batch_size=batch_size,
#     class_mode='binary',
#     shuffle=False,
# )

Found 9000 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.


In [25]:
test_dataset.labels
test_dataset.filepaths

AttributeError: 'DataFrameIterator' object has no attribute 'file_paths'

In [ ]:
metrics = [
        tf.keras.metrics.TruePositives(name = 'tp'),
        tf.keras.metrics.TrueNegatives(name = 'tn'),
        tf.keras.metrics.FalsePositives(name = 'fp'),
        tf.keras.metrics.FalseNegatives(name = 'fn'),
        tf.keras.metrics.BinaryAccuracy(name = 'acc'),
        tf.keras.metrics.Recall(name = 'r'),
        tf.keras.metrics.Precision(name = 'p'),
#         F1Score(threshold = 0.5, average='micro', name = 'F1'),
#         CohenKappa(num_classes = 2, name = 'Kappa')
    ]
epochs=10

In [ ]:
l2_strength = 1e-5

input_image = layers.Input(shape=(image_size,image_size,3))
vgg_model = VGG16(input_tensor = input_image, weights = 'imagenet', include_top=False)

for layer in vgg_model.layers:
    layer.trainable = False

x = vgg_model.output
x = layers.Flatten(input_shape=vgg_model.output_shape[1:])(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(l2_strength))(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2_strength))(x)
image_output = layers.Dense(1, activation='sigmoid')(x)

# Define the image model
image_model = Model(inputs=input_image, outputs=image_output)

image_model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=metrics)
image_model.summary()



In [ ]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(factor = 0.3)]

In [ ]:
history_fit = image_model.fit(
        train_dataset.filepaths,
    train_dataset.labels
        validation_data = val_dataset,
        batch_size = 32, 
        epochs = epochs, 
        verbose = 1, 
        callbacks = callbacks,
        shuffle = True,
        initial_epoch = 0,)

In [ ]:
# image_model.save('ImageModel/model_image.h5')
# tf.keras.utils.plot_model(image_model, "./ImageModel/model_image.png", show_shapes=True)

## BILSTM ON THE DATASET

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense



In [ ]:
# Convert the label column to strings
df_train["label"] = df_train["label"].astype(int)
df_test["label"] = df_test["label"].astype(int)
df_valid["label"] = df_valid["label"].astype(int)

In [ ]:
# Assuming df_train contains 'text' and 'label' columns
# Split the data into training and validation sets

X_train = df_train['text'].values
y_train = df_train['label'].values

X_val = df_valid['text'].values
y_val = df_valid['label'].values
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

# Pad sequences to ensure uniform length
max_length = max([len(seq) for seq in X_train_seq])
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post')


# Define the text input placeholder
text_input = layers.Input(shape=(max_length,))

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

# Pad sequences to ensure uniform length
max_length = max([len(seq) for seq in X_train_seq])
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post')


# Define the text model
embedding_layer = layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100)(text_input)
lstm_layer = layers.Bidirectional(layers.LSTM(128, return_sequences=False))(embedding_layer)
text_output = layers.Dense(1, activation='sigmoid')(lstm_layer)

text_model = Model(inputs=text_input, outputs=text_output)

# Compile the model
text_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)




In [ ]:
text_model.summary()

In [ ]:
# Train the model
text_model.fit(X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=epochs, batch_size=32)



### COMBINING THE IMAGE WITH THE TEXT MODEL

In [ ]:
train_dataset.index_array

In [ ]:

# Combine both models
combined_output = layers.concatenate([image_output, text_output])

# Additional dense layers for combined processing
x = layers.Dense(128, activation='relu')(combined_output)
output = layers.Dense(1, activation='sigmoid')(x)

# Define the combined model
combined_model = Model(inputs=[input_image, text_input], outputs=output)

# Compile the combined model
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the combined model
combined_model.summary()

## testing the combine model

In [ ]:
# history_fit = image_model.fit(
#         train_dataset, 
#         validation_data = val_dataset,
#         batch_size = 32, 
#         epochs = epochs, 
#         verbose = 1, 
#         callbacks = callbacks,
#         shuffle = True,
#         initial_epoch = 0,)

loading the test datat in the right form

In [ ]:
X_test=df_test['text'].values
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [ ]:
y_true = test_dataset.classes
y_pred = image_model.predict(test_dataset)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# Convert the predicted probabilities to binary predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred_binary)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No MISOGNY', 'mISOGYNY'], yticklabels=['No Glaucoma', 'Glaucoma'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
test_dataset

In [ ]:

predictions = combined_model.predict([test_dataset, X_test_pad], batch_size=batch_size)
predictions = predictions.reshape(predictions.shape[0])
pred = predictions > 0.5
pred = list(map(int, pred)) #true/false to 1/0

predictions_db = pd.DataFrame(data=test_df['file_name'])
predictions_db['misogynist'] = pred
